In [1]:
import cv2
import numpy as np
from PIL import Image, ImageDraw, ImageFont
import matplotlib.pyplot as plt
from pathlib import Path

In [6]:
####This cell is the older code ####
def replace_text_in_image(
    input_image_path,
    output_image_path,
    new_text_dict,
    font_path="Arial.ttf",
    text_color=(25, 25, 45),
    preview=True
):
    """
    Replace text in an image while preserving the background.
    
    Args:
        input_image_path (str): Path to input image
        output_image_path (str): Path to save modified image
        new_text_dict (dict): Dictionary with text positions and new text
        font_path (str): Path to font file
        text_color (tuple): RGB color for the new text
        preview (bool): Whether to show preview after modification
    """
    # Convert paths to Path objects for better handling
    input_path = Path(input_image_path)
    output_path = Path(output_image_path)
    
    # Verify input image exists
    if not input_path.exists():
        raise FileNotFoundError(f"Input image not found: {input_path}")
    
    # Create output directory if it doesn't exist
    output_path.parent.mkdir(parents=True, exist_ok=True)
    
    # Open the image
    img = Image.open(input_path)
    
    # Create a drawing object
    draw = ImageDraw.Draw(img)
    
    # Function to create font object with automatic size adjustment
    def get_font(text, max_width, initial_size=60):
        font_size = initial_size
        font = ImageFont.truetype(font_path, font_size)
        while font.getlength(text) > max_width and font_size > 12:
            font_size -= 1
            font = ImageFont.truetype(font_path, font_size)
        return font
    
    # Replace text for each specified position
    for text_key, (new_text, position) in new_text_dict.items():
        # Calculate appropriate font size based on text length
        max_width = 300  # Adjust based on your needs
        font = get_font(new_text, max_width)
        
        # Draw new text
        draw.text(
            position,
            new_text,
            font=font,
            fill=text_color,
            anchor="mm"  # Center alignment
        )
    
    # Save the modified image
    img.save(output_path)
    print(f"Image saved to: {output_path.absolute()}")
    
    # Show preview if requested
    if preview:
        plt.figure(figsize=(12, 8))
        
        # Show original image
        plt.subplot(1, 2, 1)
        original_img = Image.open(input_path)
        plt.imshow(original_img)
        plt.title('Original Image')
        plt.axis('off')
        
        # Show modified image
        plt.subplot(1, 2, 2)
        modified_img = Image.open(output_path)
        plt.imshow(modified_img)
        plt.title('Modified Image')
        plt.axis('off')
        
        plt.tight_layout()
        plt.show()

    return img



In [7]:
new_text_dict = {
        'name': ('JANE SMITH', (400, 200)),            # Main name
        'birth_date': ('Born June 1995', (400, 250)),  # Birth date
        'death_date': ('12 April 2024', (400, 300)),   # Death date
        'quality1': ('WISDOM', (400, 350)),            # First quality
        'quality2': ('KINDNESS', (400, 400)),          # Second quality
        'quality3': ('JOY OF LIFE', (400, 450))        # Third quality
    }

In [8]:
font_path = "C:\\Windows\\Fonts\\Arial.ttf"

In [9]:
# Replace text for each specified position
for text_key, (new_text, position) in new_text_dict.items():
    # Calculate appropriate font size based on text length
    max_width = 300  # Adjust based on your needs
    font = get_font(new_text, max_width)

NameError: name 'get_font' is not defined

In [ ]:
input_image = "media_20241030_101508_2021201713228799870.jpg"
output_image = "test_generated_image_2.png"

In [ ]:
try:
        # Call the function with all parameters
        replace_text_in_image(
            input_image_path=input_image,
            output_image_path=output_image,
            new_text_dict=new_text_dict,
            font_path=font_path,
            text_color=(25, 25, 45)  # Dark navy color
        )
        print("Image successfully modified!")
except Exception as e:
    print(f"An error occurred: {e}")

In [ ]:
#### This cell is the newer code ###


def remove_and_replace_text(
    input_image_path,
    output_image_path,
    new_text_dict=None,
    font_path="Arial.ttf",
    text_color=(25, 25, 45)
):
    """
    Remove existing text from image and optionally replace with new text.
    
    Args:
        input_image_path (str): Path to input image
        output_image_path (str): Path to save modified image
        new_text_dict (dict): Dictionary with new text and positions
        font_path (str): Path to font file
        text_color (tuple): RGB color for new text
    """
    # Read image with OpenCV
    image = cv2.imread(str(input_image_path))
    if image is None:
        raise FileNotFoundError(f"Could not read image: {input_image_path}")
    
    # Convert to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    # Create binary mask for text detection
    _, binary = cv2.threshold(gray, 240, 255, cv2.THRESH_BINARY)
    
    # Create mask for dark text
    _, dark_text = cv2.threshold(gray, 100, 255, cv2.THRESH_BINARY_INV)
    
    # Combine masks
    mask = cv2.bitwise_or(binary, dark_text)
    
    # Dilate mask to cover text regions better
    kernel = np.ones((5,5), np.uint8)
    mask = cv2.dilate(mask, kernel, iterations=2)
    
    # Invert mask for inpainting
    mask = 255 - mask
    
    # Inpaint to remove text
    result = cv2.inpaint(image, mask, 7, cv2.INPAINT_NS)
    
    # Convert back to PIL Image for text addition
    result_pil = Image.fromarray(cv2.cvtColor(result, cv2.COLOR_BGR2RGB))
    
    # Add new text if provided
    if new_text_dict:
        draw = ImageDraw.Draw(result_pil)
        
        def get_font(text, max_width, initial_size=60):
            font_size = initial_size
            font = ImageFont.truetype(font_path, font_size)
            while font.getlength(text) > max_width and font_size > 12:
                font_size -= 1
                font = ImageFont.truetype(font_path, font_size)
            return font
        
        for text_key, (new_text, position) in new_text_dict.items():
            max_width = 300
            font = get_font(new_text, max_width)
            draw.text(position, new_text, font=font, fill=text_color, anchor="mm")
    
    # Save result
    result_pil.save(output_image_path)
    
    # Show preview
    plt.figure(figsize=(15, 5))
    
    # Original image
    plt.subplot(131)
    plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    plt.title('Original Image')
    plt.axis('off')
    
    # Mask
    plt.subplot(132)
    plt.imshow(mask, cmap='gray')
    plt.title('Text Detection Mask')
    plt.axis('off')
    
    # Result
    plt.subplot(133)
    plt.imshow(result_pil)
    plt.title('Result')
    plt.axis('off')
    
    plt.tight_layout()
    plt.show()
    
    return result_pil

In [3]:
import cv2
from pathlib import Path

def extract_graphs(image_path, output_dir):
    """
    Extract individual graphs from a dashboard image and save them as separate files.
    
    Args:
        image_path (str): Path to the input dashboard image
        output_dir (str): Directory to save the extracted graphs
    """
    # Read the image
    image = cv2.imread(image_path)
    if image is None:
        raise ValueError("Could not read the image")
    
    # Convert to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    # Apply threshold to get binary image
    _, binary = cv2.threshold(gray, 240, 255, cv2.THRESH_BINARY_INV)
    
    # Find contours
    img_countours = image.copy()
    contours, hierarchy = cv2.findContours(binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    image_with_contours = cv2.drawContours(img_countours, contours, -1, (0, 255, 0), 1)

    #cv2.imshow("Contours", image_with_contours)
    #cv2.waitKey(0)  
    #cv2.destroyAllWindows()

    # Create output directory if it doesn't exist
    Path(output_dir).mkdir(parents=True, exist_ok=True)
    
    # Minimum area threshold to filter out noise
    min_area = 1000
    
    # Aspect ratio thresholds (you can adjust these based on your needs)
    min_width = 30  # Minimum acceptable width
    min_height = 30  # Minimum acceptable height
    max_aspect_ratio = 3  # Maximum width-to-height ratio for a proper rectangle/square
    min_aspect_ratio = 0.4  # Minimum acceptable width-to-height ratio (to prevent too narrow graphs)

    # Process each contour
    for idx, contour in enumerate(contours):
        # Calculate contour area
        area = cv2.contourArea(contour)
        if area > min_area:
            # Get bounding rectangle
            x, y, w, h = cv2.boundingRect(contour)
            
            # Reject graphs that are too narrow or too small
            if w < min_width or h < min_height:
                continue

            # Check the aspect ratio (width / height) to ensure it's a proper rectangle
            aspect_ratio = w / h if h != 0 else 0
            if aspect_ratio < min_aspect_ratio or aspect_ratio > max_aspect_ratio:
                continue
            
            # Add padding around the graph (optional)
            padding = 10
            x = max(0, x - padding)
            y = max(0, y - padding)
            w = min(image.shape[1] - x, w + 2 * padding)
            h = min(image.shape[0] - y, h + 2 * padding)
            
            # Extract the graph region
            graph = image[y:y+h, x:x+w]
            
            # Save the extracted graph
            output_path = f"{output_dir}/graph_{idx}.png"
            cv2.imwrite(output_path, graph)
            print(f"Saved graph {idx} to {output_path}")


In [5]:
image_path = "img/Predictive_Modeling_Model_Performance_1.jpg"
output_dir = "out"

extract_graphs(image_path, output_dir)


Saved graph 141 to out/graph_141.png
Saved graph 142 to out/graph_142.png
Saved graph 145 to out/graph_145.png
Saved graph 146 to out/graph_146.png
